# TEST PLAYGROUND
#Just for dev puproses!


# Austins code for loss function:

In [2]:
def penalized_loss_fn(lam):
    def loss(y_true, y_pred):

        """The y_true term contains the regularizor HyCOM step on the second axis.
        This is not directly generated by the model, so we slice the term off the end
        and simply use it as a regularization term to the buoy forecast.

        lam = 0 is no real buoy data used, all HyCOM regularization
        lam = 1 is all real buoy data, no HyCOM regularization"""

        print(y_true.shape, y_pred.shape)

        water_temp_true = y_true[:,0]
        hy_water_temp_true = y_true[:,3]

        water_delta1 = y_pred[:,0] - water_temp_true
        water_delta2 = y_pred[:,0] - hy_water_temp_true

        gust_temp_true = y_true[:,1]
        c_gust_temp_true = y_true[:,11]

        gust_delta1 = y_pred[:,1] - gust_temp_true
        gust_delta2 = y_pred[:,1] - c_gust_temp_true

        pres_temp_true = y_true[:,2]
        c_pres_temp_true = y_true[:,15]

        pres_delta1 = y_pred[:,2] - pres_temp_true
        pres_delta2 = y_pred[:,2] - c_pres_temp_true


        model_pred = tf.math.reduce_mean(keras.backend.abs(y_pred[:,3:]-y_true[:,3:]))

        temp_out =  tf.math.reduce_mean( tf.math.scalar_mul(lam, keras.backend.abs(water_delta1))
                                  + tf.math.scalar_mul((1-lam), keras.backend.abs(water_delta2)))

        gust_out = tf.math.reduce_mean( tf.math.scalar_mul(lam, keras.backend.abs(gust_delta1))
                                  + tf.math.scalar_mul((1-lam), keras.backend.abs(gust_delta2)))

        pres_out = tf.math.reduce_mean( tf.math.scalar_mul(lam, keras.backend.abs(pres_delta1))
                                  + tf.math.scalar_mul((1-lam), keras.backend.abs(pres_delta2)))

        return temp_out+gust_out+pres_out+model_pred

    return loss

# Austins code for Transformer Model:

lam = alpha

In [3]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.05):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads,
                                             key_dim=embed_dim)
        self.ffn = keras.Sequential([
            layers.Dense(ff_dim, activation="selu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)  # self-attention layer
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)  # layer norm
        ffn_output = self.ffn(out1)  #feed-forward layer
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)  # layer norm

def build_attention_model(lam, num_channels=18, deep_layers=4):
    #embed_dim = n_features  # Embedding size for each token
    embed_dim = 512
    num_heads = 10  # Number of attention heads
    ff_dim = 500  # Hidden layer size in feed forward network inside transformer

    model = tf.keras.Sequential()
    model.add(layers.Reshape(target_shape=(1,num_channels)))
    model.add(layers.Dense(512) )
    model.add(layers.BatchNormalization( scale=True, center=True, momentum=0.999) )

    for a in range(deep_layers):
        model.add(TransformerBlock(embed_dim, num_heads, ff_dim, rate=0.05))
        model.add(layers.Dropout(0.05))
        model.add(layers.LSTM(512, return_sequences=True))
        model.add(layers.Dropout(0.05))
        model.add(layers.Dense(512) )
        model.add(layers.Dropout(0.05))
        model.add( layers.BatchNormalization( scale=True, center=True, momentum=0.999) )

    for a in range(4):
            model.add(layers.Dense(200, ))
            model.add(layers.Dropout(0.05))

    model.add(keras.layers.Flatten(name='Flatten'))

    model.add(layers.Dense(num_channels, activation='linear'))

    learning_rate = 1e-6
    decay_rate = 1e-10
    momentum = 0.9

    sgd = keras.optimizers.legacy.Adam(learning_rate=learning_rate,  decay=decay_rate, ) #stochastic gradient decent ?!

    model.compile(loss=penalized_loss_fn(lam), optimizer=sgd)

    return model

model=build_attention_model(lam=0.4,
                            num_channels=18,
                            deep_layers=1
                            )

model.build((None,18,1)) # `input_shape` is the shape of the input data
                         # e.g. input_shape = (None, 32, 32, 3)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_1 (Reshape)         (None, 1, 18)             0         
                                                                 
 dense_9 (Dense)             (None, 1, 512)            9728      
                                                                 
 batch_normalization_2 (Batc  (None, 1, 512)           2048      
 hNormalization)                                                 
                                                                 
 transformer_block_1 (Transf  (None, 1, 512)           11016692  
 ormerBlock)                                                     
                                                                 
 dropout_11 (Dropout)        (None, 1, 512)            0         
                                                                 
 lstm_1 (LSTM)               (None, 1, 512)           